In [6]:
import gym
import copy
import torch
from torch.autograd import Variable
import random
import matplotlib.pyplot as plt
from PIL import Image
# from IPython.display import clear_output
import math
import torchvision.transforms as T
import numpy as np

import time

class DQN():
    ''' Deep Q Neural Network class. '''
    def __init__(self, state_dim, action_dim, hidden_dim=64, lr=0.05):
            self.criterion = torch.nn.MSELoss()
            self.model = torch.nn.Sequential(
                            torch.nn.Linear(state_dim, hidden_dim),
                            torch.nn.LeakyReLU(),
                            torch.nn.Linear(hidden_dim, hidden_dim*2),
                            torch.nn.LeakyReLU(),
                            torch.nn.Linear(hidden_dim*2, action_dim)
                    )
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr)



    def update(self, state, y):
        """Update the weights of the network given a training sample. """
        y_pred = self.model(torch.Tensor(state))
        loss = self.criterion(y_pred, Variable(torch.Tensor(y)))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


    def predict(self, state):
        """ Compute Q values for all actions using the DQL. """
        with torch.no_grad():
            print(state)
            return self.model(torch.Tensor(state))
        
        
        
        
def q_learning(env, model, episodes, gamma=0.9, 
               epsilon=0.3, eps_decay=0.99,
               replay=False, replay_size=20, 
               title = 'DQL', double=False, 
               n_update=10, soft=False, verbose=True):
    """Deep Q Learning algorithm using the DQN. """
    final = []
    memory = []
    episode_i=0
    sum_total_replay_time=0
    for episode in range(episodes):
        episode_i+=1
        if double and not soft:
            # Update target network every n_update steps
            if episode % n_update == 0:
                model.target_update()
        if double and soft:
            model.target_update()
        
        # Reset state
        state = env.reset()
        done = False
        total = 0
        
        while not done:
            # Implement greedy search policy to explore the state space
            if random.random() < epsilon:
                action = env.action_space.sample()
            else:
                q_values = model.predict(state)
                action = torch.argmax(q_values).item()
            
            # Take action and add reward to total
            next_state, reward, done, _ = env.step(action)
            
            # Update total and memory
            total += reward
            memory.append((state, action, next_state, reward, done))
            q_values = model.predict(state).tolist()
            if done:
                if not replay:
                    q_values[action] = reward
                    # Update network weights
                    model.update(state, q_values)
                break

            if replay:
                t0=time.time()
                # Update network weights using replay memory
                model.replay(memory, replay_size, gamma)
                t1=time.time()
                sum_total_replay_time+=(t1-t0)
            else: 
                # Update network weights using the last step only
                q_values_next = model.predict(next_state)
                q_values[action] = reward + gamma * torch.max(q_values_next).item()
                model.update(state, q_values)

            state = next_state
        
        # Update epsilon
        epsilon = max(epsilon * eps_decay, 0.01)
        final.append(total)
#         plot_res(final, title)
        
        if verbose:
            print("episode: {}, total reward: {}".format(episode_i, total))
            if replay:
                print("Average replay time:", sum_total_replay_time/episode_i)
                
#         if episode % 10==0:
            
        
    return final



# Expand DQL class with a replay function.
class DQN_replay(DQN):
    #old replay function
    #def replay(self, memory, size, gamma=0.9):
        #""" Add experience replay to the DQN network class. """
        # Make sure the memory is big enough
        #if len(memory) >= size:
            #states = []
            #targets = []
            # Sample a batch of experiences from the agent's memory
            #batch = random.sample(memory, size)
            
            # Extract information from the data
            #for state, action, next_state, reward, done in batch:
                #states.append(state)
                # Predict q_values
                #q_values = self.predict(state).tolist()
                #if done:
                    #q_values[action] = reward
                #else:
                    #q_values_next = self.predict(next_state)
                    #q_values[action] = reward + gamma * torch.max(q_values_next).item()
     
                #targets.append(q_values)

            #self.update(states, targets)
    
    #new replay function
    def replay(self, memory, size, gamma=0.9):
        """New replay function"""
        #Try to improve replay speed
        if len(memory)>=size:
            batch = random.sample(memory,size)
            batch_t = list(map(list, zip(*batch))) #Transpose batch list
            states = batch_t[0]
            actions = batch_t[1]
            next_states = batch_t[2]
            rewards = batch_t[3]
            is_dones = batch_t[4]
        
            states = torch.Tensor(states)
            actions_tensor = torch.Tensor(actions)
            next_states = torch.Tensor(next_states)
            rewards = torch.Tensor(rewards)
            is_dones_tensor = torch.Tensor(is_dones)
        
            is_dones_indices = torch.where(is_dones_tensor==True)[0]
        
            all_q_values = self.model(states) # predicted q_values of all states
            all_q_values_next = self.model(next_states)
            #Update q values
            all_q_values[range(len(all_q_values)),actions]=rewards+gamma*torch.max(all_q_values_next, axis=1).values
            all_q_values[is_dones_indices.tolist(), actions_tensor[is_dones].tolist()]=rewards[is_dones_indices.tolist()]
        
            
            self.update(states.tolist(), all_q_values.tolist())
            
            
            
            



In [7]:
env = gym.envs.make("CartPole-v1")
# Number of states
n_state = env.observation_space.shape[0]
# Number of actions
n_action = env.action_space.n
# Number of episodes
episodes = 200
# Number of hidden nodes in the DQN
n_hidden = 50
# Learning rate
lr = 0.001
# Get replay results


In [8]:
dqn_replay = DQN_replay(n_state, n_action, n_hidden, lr)
replay = q_learning(env, dqn_replay, 
                    episodes, gamma=.9, 
                    epsilon=0.2, replay=True, 
                    title='DQL with Replay')

[-0.00441332  0.00614838 -0.00291587 -0.0095407 ]
[-0.00441332  0.00614838 -0.00291587 -0.0095407 ]
[-0.00429035 -0.18893164 -0.00310669  0.28222081]
[-0.00429035 -0.18893164 -0.00310669  0.28222081]
[-0.00806899  0.00623449  0.00253773 -0.01144033]
[-0.00806899  0.00623449  0.00253773 -0.01144033]
[-0.0079443  -0.18892376  0.00230892  0.2820422 ]
[-0.0079443  -0.18892376  0.00230892  0.2820422 ]
[-0.01172277  0.00616518  0.00794977 -0.00991161]
[-0.01159947  0.20117222  0.00775154 -0.30007572]
[-0.01159947  0.20117222  0.00775154 -0.30007572]
[-0.00757603  0.00594065  0.00175002 -0.00495823]
[-0.00757603  0.00594065  0.00175002 -0.00495823]
[-0.00745721 -0.18920636  0.00165086  0.28827634]
[-0.01124134  0.00589201  0.00741638 -0.00388547]
[-0.01124134  0.00589201  0.00741638 -0.00388547]
[-0.0111235  -0.18933551  0.00733867  0.29112816]
[-0.0111235  -0.18933551  0.00733867  0.29112816]
[-0.01491021  0.00568103  0.01316124  0.00076877]
[-0.01491021  0.00568103  0.01316124  0.00076877]


[-0.07654645 -0.93453897  0.19397551  1.62528909]
[-0.07654645 -0.93453897  0.19397551  1.62528909]
episode: 7, total reward: 10.0
Average replay time: 0.03446224757603237
[ 0.00452743  0.01663519 -0.03483219 -0.0433406 ]
[ 0.00452743  0.01663519 -0.03483219 -0.0433406 ]
[ 0.00486013 -0.17797042 -0.035699    0.23815215]
[ 0.00486013 -0.17797042 -0.035699    0.23815215]
[ 0.00130073 -0.37256466 -0.03093596  0.51936427]
[ 0.00130073 -0.37256466 -0.03093596  0.51936427]
[-0.00615057 -0.56723774 -0.02054867  0.80214044]
[-0.00615057 -0.56723774 -0.02054867  0.80214044]
[-0.01749532 -0.76207196 -0.00450586  1.08828918]
[-0.01749532 -0.76207196 -0.00450586  1.08828918]
[-0.03273676 -0.56689088  0.01725992  0.79419582]
[-0.03273676 -0.56689088  0.01725992  0.79419582]
[-0.04407458 -0.76224544  0.03314384  1.09225818]
[-0.05931949 -0.95778809  0.054989    1.39515357]
[-0.05931949 -0.95778809  0.054989    1.39515357]
[-0.07847525 -0.76339177  0.08289207  1.12015846]
[-0.07847525 -0.76339177  0.

[-0.01877105 -0.20811374  0.06418286  0.44499801]
[-0.02293333 -0.01395579  0.07308282  0.17321755]
[-0.02293333 -0.01395579  0.07308282  0.17321755]
[-0.02321244 -0.21004349  0.07654717  0.48803119]
[-0.02321244 -0.21004349  0.07654717  0.48803119]
[-0.02741331 -0.01608028  0.08630779  0.22042125]
[-0.02741331 -0.01608028  0.08630779  0.22042125]
[-0.02773492 -0.21232309  0.09071622  0.53903278]
[-0.03198138 -0.40859538  0.10149687  0.85886464]
[-0.03198138 -0.40859538  0.10149687  0.85886464]
[-0.04015329 -0.21499158  0.11867417  0.59974235]
[-0.04015329 -0.21499158  0.11867417  0.59974235]
[-0.04445312 -0.02171235  0.13066901  0.34667102]
[-0.04445312 -0.02171235  0.13066901  0.34667102]
[-0.04488736  0.17133234  0.13760243  0.09788002]
[-0.04488736  0.17133234  0.13760243  0.09788002]
[-0.04146072 -0.02546599  0.13956003  0.4306148 ]
[-0.04146072 -0.02546599  0.13956003  0.4306148 ]
[-0.04197004  0.16743239  0.14817233  0.18497806]
[-0.04197004  0.16743239  0.14817233  0.18497806]


[-0.03025441  0.02900019  0.04956604  0.06852695]
[-0.03025441  0.02900019  0.04956604  0.06852695]
[-0.02967441  0.22337777  0.05093658 -0.20811488]
[-0.02520685  0.41773577  0.04677428 -0.48430488]
[-0.02520685  0.41773577  0.04677428 -0.48430488]
[-0.01685214  0.61216748  0.03708818 -0.76188685]
[-0.01685214  0.61216748  0.03708818 -0.76188685]
[-0.00460879  0.80675944  0.02185044 -1.04267264]
[-0.00460879  0.80675944  0.02185044 -1.04267264]
[ 1.15264016e-02  1.00158449e+00  9.96990163e-04 -1.32841694e+00]
[ 1.15264016e-02  1.00158449e+00  9.96990163e-04 -1.32841694e+00]
[ 0.03155809  0.80644998 -0.02557135 -1.03542219]
[ 0.03155809  0.80644998 -0.02557135 -1.03542219]
[ 0.04768709  1.0019024  -0.04627979 -1.33602214]
[ 0.06772514  0.80739312 -0.07300024 -1.05817202]
[ 0.06772514  0.80739312 -0.07300024 -1.05817202]
[ 0.083873    1.00340232 -0.09416368 -1.37284577]
[ 0.083873    1.00340232 -0.09416368 -1.37284577]
[ 0.10394105  0.80957534 -0.12162059 -1.11103654]
[ 0.10394105  0.80

[-0.20091035 -0.56841321 -0.09375392 -0.1243518 ]
[-0.21227861 -0.76207568 -0.09624095  0.13734286]
[-0.21227861 -0.76207568 -0.09624095  0.13734286]
[-0.22752012 -0.56571644 -0.09349409 -0.18408418]
[-0.22752012 -0.56571644 -0.09349409 -0.18408418]
[-0.23883445 -0.75938493 -0.09717578  0.07770258]
[-0.23883445 -0.75938493 -0.09717578  0.07770258]
[-0.25402215 -0.95298921 -0.09562173  0.33821417]
[-0.25402215 -0.95298921 -0.09562173  0.33821417]
[-0.27308194 -0.75664586 -0.08885744  0.01697461]
[-0.27308194 -0.75664586 -0.08885744  0.01697461]
[-0.28821485 -0.95038856 -0.08851795  0.28035239]
[-0.28821485 -0.95038856 -0.08851795  0.28035239]
[-0.30722262 -0.75412276 -0.0829109  -0.038884  ]
[-0.30722262 -0.75412276 -0.0829109  -0.038884  ]
[-0.32230508 -0.94796393 -0.08368858  0.22653107]
[-0.32230508 -0.94796393 -0.08368858  0.22653107]
[-0.34126436 -1.14179635 -0.07915796  0.49168638]
[-0.34126436 -1.14179635 -0.07915796  0.49168638]
[-0.36410029 -0.94565228 -0.06932423  0.17514344]


...

[ 0.303611   -0.23804752 -0.12444667  0.1643966 ]
[ 0.303611   -0.23804752 -0.12444667  0.1643966 ]
[ 0.29885005 -0.04138417 -0.12115874 -0.16481003]
[ 0.29885005 -0.04138417 -0.12115874 -0.16481003]
[ 0.29802237 -0.23458219 -0.12445494  0.08732898]
[ 0.29333073 -0.03791641 -0.12270836 -0.24188339]
[ 0.2925724  -0.23109138 -0.12754603  0.00971448]
[ 0.2925724  -0.23109138 -0.12754603  0.00971448]
[ 0.28795057 -0.42417532 -0.12735174  0.25959084]
[ 0.28795057 -0.42417532 -0.12735174  0.25959084]
[ 0.27946706 -0.22748715 -0.12215992 -0.07039245]
[ 0.27946706 -0.22748715 -0.12215992 -0.07039245]
[ 0.27491732 -0.42066523 -0.12356777  0.18138949]
[ 0.26650402 -0.22401167 -0.11993998 -0.14757823]
[ 0.26650402 -0.22401167 -0.11993998 -0.14757823]
[ 0.26202378 -0.41723009 -0.12289154  0.10498935]
[ 0.26202378 -0.41723009 -0.12289154  0.10498935]
[ 0.25367918 -0.61039622 -0.12079176  0.35651411]
[ 0.24147126 -0.8036123  -0.11366148  0.60879997]
[ 0.24147126 -0.8036123  -0.11366148  0.60879997]


[-0.4574284  -0.24317047 -0.02687833  0.27616345]
[-0.46229181 -0.04767556 -0.02135506 -0.02487427]
[-0.46229181 -0.04767556 -0.02135506 -0.02487427]
[-0.46324532 -0.24248486 -0.02185254  0.26099513]
[-0.46324532 -0.24248486 -0.02185254  0.26099513]
[-0.46809502 -0.04705789 -0.01663264 -0.03849947]
[-0.46809502 -0.04705789 -0.01663264 -0.03849947]
[-0.46903618 -0.24193743 -0.01740263  0.24888963]
[-0.46903618 -0.24193743 -0.01740263  0.24888963]
[-0.47387493 -0.04657133 -0.01242484 -0.04923126]
[-0.47387493 -0.04657133 -0.01242484 -0.04923126]
[-0.47480635 -0.24151293 -0.01340946  0.23950577]
[-0.47480635 -0.24151293 -0.01340946  0.23950577]
[-0.47963661 -0.04620201 -0.00861935 -0.05737653]
[-0.47963661 -0.04620201 -0.00861935 -0.05737653]
[-0.48056065 -0.24119933 -0.00976688  0.23257451]
[-0.48056065 -0.24119933 -0.00976688  0.23257451]
[-0.48538464 -0.04593919 -0.00511539 -0.06317314]
[-0.48538464 -0.04593919 -0.00511539 -0.06317314]
[-0.48630342  0.14925573 -0.00637885 -0.3574656 ]


[-0.7821823  -0.22813793  0.03671991 -0.05559934]
[-0.7821823  -0.22813793  0.03671991 -0.05559934]
[-0.78674506 -0.42376663  0.03560792  0.24843912]
[-0.78674506 -0.42376663  0.03560792  0.24843912]
[-0.79522039 -0.22917082  0.04057671 -0.03280305]
[-0.79522039 -0.22917082  0.04057671 -0.03280305]
[-0.79980381 -0.03465353  0.03992065 -0.31241253]
[-0.79980381 -0.03465353  0.03992065 -0.31241253]
[-0.80049688 -0.23032079  0.03367239 -0.00741192]
[-0.80049688 -0.23032079  0.03367239 -0.00741192]
[-0.8051033  -0.03569753  0.03352416 -0.28928352]
[-0.8051033  -0.03569753  0.03352416 -0.28928352]
[-0.80581725 -0.23128109  0.02773849  0.01378123]
[-0.80581725 -0.23128109  0.02773849  0.01378123]
[-0.81044287 -0.0365677   0.02801411 -0.27002243]
[-0.81044287 -0.0365677   0.02801411 -0.27002243]
[-0.81117422 -0.23207799  0.02261366  0.03136301]
[-0.81117422 -0.23207799  0.02261366  0.03136301]
[-0.81581578 -0.0372875   0.02324092 -0.25410016]
[-0.81581578 -0.0372875   0.02324092 -0.25410016]


[-1.74736652 -0.97092175 -0.05342567  0.28390604]
[-1.74736652 -0.97092175 -0.05342567  0.28390604]
[-1.76678495 -0.77508012 -0.04774754 -0.02513694]
[-1.76678495 -0.77508012 -0.04774754 -0.02513694]
[-1.78228655 -0.96948596 -0.04825028  0.25210704]
[-1.78228655 -0.96948596 -0.04825028  0.25210704]
[-1.80167627 -1.16388691 -0.04320814  0.52918901]
[-1.82495401 -1.35837521 -0.03262436  0.80794994]
[-1.82495401 -1.35837521 -0.03262436  0.80794994]
[-1.85212151 -1.16282171 -0.01646536  0.50518591]
[-1.85212151 -1.16282171 -0.01646536  0.50518591]
[-1.87537795 -0.96747164 -0.00636165  0.20735993]
[-1.87537795 -0.96747164 -0.00636165  0.20735993]
[-1.89472738 -1.16250204 -0.00221445  0.49802931]
[-1.89472738 -1.16250204 -0.00221445  0.49802931]
[-1.91797742 -0.96734894  0.00774614  0.20464932]
[-1.91797742 -0.96734894  0.00774614  0.20464932]
[-1.9373244  -1.1625808   0.01183913  0.49976568]
[-1.9373244  -1.1625808   0.01183913  0.49976568]
[-1.96057602 -0.96762774  0.02183444  0.2108372 ]


KeyboardInterrupt: 